In [31]:
from pymongo import MongoClient
from bs4 import BeautifulSoup as bs
import requests
import re
from pprint import pprint

### 1 Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта hh.ru

In [317]:
def query(link,params={}):
    """
    Функция содержит общие параметры необходимые для запроса. 
    link – хранит ссылку на сайт, на который адресован запрос.
    params  - необязательный параметр, содержит передаваемые параметры, параметры можно передать и в ссылке link, а params оставить не заполненным.
    Возвращает функция DOM дерево.
    """
    header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
          'Accept':'*/*'}
    if params:
        response = requests.get(link,headers=header,params=params)
        print(response.url)
        response = response.text
    else:
        response = requests.get(link,headers=header).text
   
    soup = bs(response,'lxml')    
    return soup



def getSalary(s):
    
    """
    Функция обрабатывает строки содержащие величину заработной платы и возвращает dist, содержащий цены в структурированном виде.
    """
    
    obj = {"min":None,
           "max":None,
           "currency_unit":None
          }
    if not s:
        return obj
    q = re.sub('\s*', "", s)
    result = re.findall(r'([^0-9]*)(?:(\d*)\W)*(\d+)(\w+)', q)
    if result:
        result = result[0]
        obj["currency_unit"] = result[3].upper()
        if result[0].upper() == "от".upper():        
            obj["min"] = float(result[2])
        elif result[0].upper() == "до".upper():      
            obj["max"] = float(result[2])
        elif result[0] == "":
            obj["min"] = float(result[1])
            obj["max"] = float(result[2])           
    return obj
    


In [318]:
main_link = 'https://murmansk.hh.ru/search/vacancy?text=data+scientist'
base_link = 'https://murmansk.hh.ru'
params = {'L_is_autosearch':'false',
            'clusters':'true',
            'enable_snippets':'true',           
            'page':0}

In [319]:
hh = []
while True:
    soup = query(link=main_link,params=params)
    vacancy_serpclass = soup.find('div', {'class':'vacancy-serp'})
    vacancies = vacancy_serpclass.select(".vacancy-serp-item")
    pprint(len(vacancies))    
    button = soup.find('a', {"data-qa":"pager-next"})#кнопка ДАЛЕЕ    
    
    for item in vacancies:
        a = item.find('a', {"data-qa":"vacancy-serp__vacancy-title"})
        site = item.find('a', {"data-qa":"vacancy-serp__vacancy-employer"})        
        salary = item.find('span', {"data-qa":"vacancy-serp__vacancy-compensation"})
        salary = salary.text if salary else None
        salary = getSalary(salary)
        obj = {'specialty':a.string,
               'salary_max':salary['max'],
               'salary_min':salary['min'],
               'salary_unit':salary['currency_unit'],
               'link':a.get("href"),
               'site':base_link+site.get("href") if site else None,
               'site_job':'HH',
               '_id':str(re.findall(r'/(\d+)\?', a.get("href"))[0])+'hh'}        
        f = False
        for item in hh:
            if item['_id'] == obj['_id']:
                f = True
                break        
        if not f:
            hh.append(obj)
        
    if not button:  #проверяю наличие, кнопку ДАЛЕЕ
        break
    
    params["page"] = params["page"] + 1

https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=0
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=1
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=2
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=3
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=4
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=5
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=6
39


### 2 Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru

In [320]:
main_link = 'https://russia.superjob.ru/vakansii/programmist-python.html'
base_link = 'https://russia.superjob.ru'
params = {'page':1}
soup = query(link=main_link,params=params)

https://russia.superjob.ru/vakansii/programmist-python.html?page=1


In [322]:
superjob = []
while True:
    soup = query(link=main_link,params=params)
    vacancies = soup.select(".iJCa5.f-test-vacancy-item._1fma_._1JhPh._2gFpt._1znz6._2nteL")
    pprint(len(vacancies)) 
    button = soup.select('.icMQ_._1_Cht._3ze9n.f-test-button-dalshe.f-test-link-Dalshe')#кнопка ДАЛЕЕ   
    for item in vacancies:
        a = item.select(".icMQ_._6AfZ9")[0]        
        site = item.select('.icMQ_._205Zx')
        site = site[0] if site else None        
        salary = item.select('._3mfro._2Wp8I.PlM3e._2JVkc._2VHxz')
        salary = salary[0].text if salary else None
        salary = getSalary(salary)        
        obj = {'specialty':a.text,
               'salary_max':salary['max'],
               'salary_min':salary['min'],
               'salary_unit':salary['currency_unit'],
               'link':base_link+a.get("href"),
               'site':base_link+site.get("href") if site else None,
               'site_job':'superjob',
               '_id':str(re.findall(r'-(\d+)\.', a.get("href"))[0])+'superjob'}
        f = False
        for item in superjob:
            if item['_id'] == obj['_id']:
                f = True
                break        
        if not f:            
            superjob.append(obj)
    if not button:#проверяю наличие, кнопку ДАЛЕЕ
        break
    params["page"] = params["page"] + 1

https://russia.superjob.ru/vakansii/programmist-python.html?page=1
20
https://russia.superjob.ru/vakansii/programmist-python.html?page=2
4


# Lesson 3

In [342]:
client = MongoClient('localhost', 27017)
db = client['vacancies']

### 3. Написать  функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.

In [363]:
def add_new_vacancies(obj_vacancies, obj_base):
    '''
    Функция принимает список вакансий dist. и объект базы данных, в которую будут сохраняться вакансии.
    '''
    res_vacancies = [s['_id'] for s in obj_vacancies]
    base_vacancies = list(obj_base.find({}))    
    res_base = [s['_id'] for s in base_vacancies]
    res_vacancies_set = set(res_vacancies)
    res_base_set = set(res_base)
    new_vacancies = res_vacancies_set - res_base_set
    
   
    res_base_vacancies = []
    res_superjob = []
    
    for item in obj_vacancies:        
        if item["_id"] in new_vacancies:            
            res_base_vacancies.append(item)
    if res_base_vacancies:
        obj_base.insert_many(res_base_vacancies)
        print(f"В базу добавленно {len(res_base_vacancies)} записей")


In [366]:
#db.hh.delete_many({})
#db.superjob.delete_many({})

### 1. реализовать функцию, записывающую собранные вакансии в созданную БД

In [367]:
add_new_vacancies(hh,db.hh)

В базу добавленно 297 записей


In [368]:
add_new_vacancies(superjob,db.superjob)

В базу добавленно 24 записей


### 2. Написать  функцию,  которая  производит  поиск  и  выводит  на  экран  вакансии  с  заработной платой больше введённой суммы

In [ ]:
def get_wage(num): 
    '''
    функция, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы num
    '''
    guery = {"$or":[{"salary_min":{'$gt':num}},{'salary_min':{"$gt":num}}]}
    hh = list(db.hh.find(guery))
    superjob =list(db.superjob.find(guery))
    return hh + superjob
    for item in hh:
        pprint(item)
        

In [ ]:
get_wage(100000)